Requirements:

* **PROJECT_ID** variable set - from Google Cloud Console
* **credntials.json** file present - downloaded from Google Cloud Console after service account creation
* **requirements.txt** installed

In [6]:
from google.cloud import translate
from google.oauth2 import service_account

PROJECT_ID = 'extreme-water-298017'
credentials = service_account.Credentials.from_service_account_file('credentials.json')
parent = f"projects/{PROJECT_ID}/locations/global"

translate_client = translate.TranslationServiceClient(credentials=credentials)

sampleTexts = [
    
    'Ważna informacja. Nikt nie będzie szczepił na świecie dzieci poniżej 16 roku życia. \
    Żadna szczepionka nie będzie przygotowana dla tej grupy wiekowej. Dlaczego?\
    Ponieważ według WHO koronawirus prawie nie dotyczy dzieci. W takim razie dlaczego u nas szkoły są ciągle zamknięte?',
    
    'Mamy 13 239 nowych i potwierdzonych przypadków zakażenia #koronawirus z województw: mazowieckiego (1643), \
    wielkopolskiego (1380), śląskiego (1355), kujawsko-pomorskiego (1081), zachodniopomorskiego (999),\
    warmińsko-mazurskiego (964), łódzkiego (915), dolnośląskiego (828).',
    
    '@MWosPL: "Nam się udało. Zakaziliśmy się #koronawirus i wróciliśmy do zdrowia.\
    Teraz w naszej krwi, w osoczu, są przeciwciała, które podane chorym na #Covid19 mogą uratować ich życie...',
    
    'Dr.Sucharit Bhakdi na FOX-NEWS. INGRAHAM: Więc myśli Pan, że szczepionka COVID nie jest konieczna?\
    Dr.BHAKDI: Myślę,że to niezwykle niebezpieczne.I ostrzegam, jeśli pójdziecie w tym kierunku. \
    Szczep siebie i swoją rodzinę @a_niedzielski!!! A od nas wara! #Covid19 #Plandemia',
        
]

response = translate_client.translate_text(request = {
    "parent": parent,
    "contents": sampleTexts,
    "source_language_code": 'pl-PL',
    "target_language_code": 'en-US',
    "mime_type": 'text/plain'
})
translations = [r.translated_text for r in response.translations]
for o, t in zip(sampleTexts, translations):
    print(o, '\n---\n', t, '\n\n')

Ważna informacja. Nikt nie będzie szczepił na świecie dzieci poniżej 16 roku życia.     Żadna szczepionka nie będzie przygotowana dla tej grupy wiekowej. Dlaczego?    Ponieważ według WHO koronawirus prawie nie dotyczy dzieci. W takim razie dlaczego u nas szkoły są ciągle zamknięte? 
---
 Important information. Nobody in the world will vaccinate children under the age of 16. No vaccine will be prepared for this age group. Why? Because according to WHO, the coronavirus hardly affects children. So why are our schools still closed? 


Mamy 13 239 nowych i potwierdzonych przypadków zakażenia #koronawirus z województw: mazowieckiego (1643),     wielkopolskiego (1380), śląskiego (1355), kujawsko-pomorskiego (1081), zachodniopomorskiego (999),    warmińsko-mazurskiego (964), łódzkiego (915), dolnośląskiego (828). 
---
 We have 13,239 new and confirmed cases of #coronavirus infection from the following voivodeships: Mazowieckie (1643), Wielkopolskie (1380), Śląskie (1355), Kujawsko-Pomorskie (1

DocumentSentiment contains the overall sentiment of the document, which consists of the following fields:
**score of the sentiment ranges between -1.0 (negative) and 1.0 (positive)** and corresponds to the overall emotional leaning of the text.

**magnitude indicates the overall strength of emotion (both positive and negative) within the given text, between 0.0 and +inf**. Unlike score, magnitude is not normalized; each expression of emotion within the text (both positive and negative) contributes to the text's magnitude (so longer text blocks may have greater magnitudes).

The **score** of a document's sentiment indicates the overall emotion of a document. 

The **magnitude** of a document's sentiment indicates how much emotional content is present within the document, and this value is often proportional to the length of the document.

A document with a neutral score (around 0.0) may indicate a low-emotion document, or may indicate mixed emotions, with both high positive and negative values which cancel each out. Generally, you can use magnitude values to disambiguate these cases, as truly neutral documents will have a low magnitude value, while mixed documents will have higher magnitude values.

In [7]:
from google.cloud import language_v1

client = language_v1.LanguageServiceClient(credentials=credentials)

type = language_v1.Document.Type.PLAIN_TEXT
language = "en"
encoding_type = language_v1.EncodingType.UTF8

for text in translations:
    document = {"content": text, "type_": type, "language": language}
    response = client.analyze_sentiment(request = {'document': document, 'encoding_type': encoding_type})
    print(u"Document \n\t score: {} \n\t magnitude: {}\n".format(
        response.document_sentiment.score, response.document_sentiment.magnitude)
         )
    print("Sentences \n")
    for sentence in response.sentences:
        print(u"\ttext: {}".format(sentence.text.content))
        print(u"\tscore: {}".format(sentence.sentiment.score))
        print(u"\tmagnitude: {}\n".format(sentence.sentiment.magnitude))
    print('\n---------\n')

Document 
	 score: -0.30000001192092896 
	 magnitude: 3.4000000953674316

Sentences 

	text: Important information.
	score: 0.6000000238418579
	magnitude: 0.6000000238418579

	text: Nobody in the world will vaccinate children under the age of 16.
	score: -0.6000000238418579
	magnitude: 0.6000000238418579

	text: No vaccine will be prepared for this age group.
	score: -0.5
	magnitude: 0.5

	text: Why?
	score: -0.5
	magnitude: 0.5

	text: Because according to WHO, the coronavirus hardly affects children.
	score: -0.20000000298023224
	magnitude: 0.20000000298023224

	text: So why are our schools still closed?
	score: -0.699999988079071
	magnitude: 0.699999988079071


---------

Document 
	 score: -0.10000000149011612 
	 magnitude: 0.10000000149011612

Sentences 

	text: We have 13,239 new and confirmed cases of #coronavirus infection from the following voivodeships: Mazowieckie (1643), Wielkopolskie (1380), Śląskie (1355), Kujawsko-Pomorskie (1081), Zachodniopomorskie (999), Warmińsko-Maz